In [2]:
# import twint
import pandas as pd
import os, calendar

# import nest_asyncio
# nest_asyncio.apply()

In [7]:
df = pd.read_pickle('preprocess.pkl')
df.head()

,id,date,content,replyCount,likeCount,quoteCount,sourceLabel,user_username,user_verified,user_created,user_followersCount,user_friendsCount,user_statusesCount,mentionedUsers_account,mentionedUsers_count,account,bank,tidy_twits
0,1222787293951315969,2020-01-30 07:43:03+00:00,@promo_BRI Ratas saldo apaan min ??,1,0,0,Twitter for Android,Tenshi_Yeol,False,2010-09-29T09:56:22+00:00,203,55,49059,None,0,promo_bri,BRI,"[ratas, saldo, min]"
1,1222779502398332928,2020-01-30 07:12:06+00:00,BRI Setujui Restrukturisasi Utang Krakatau Ste...,0,0,0,Twitter Web App,KoranBUMN,False,2016-12-22T11:32:50+00:00,746,433,37476,"[kontakBRI, BANKBRI_ID, KemenBUMN, ojkindonesia]",4,promo_bri,BRI,"[bri, tuju, restrukturisasi, utang, krakatau, ..."
2,1222745137475751937,2020-01-30 04:55:32+00:00,@promo_BRI min mau nge blokir atm hilang,3,0,0,Twitter for Android,Sayyaf12,False,2011-06-06T04:32:49+00:00,316,288,2898,None,0,promo_bri,BRI,"[min, nge, blokir, atm, hilang]"
3,1222699656813006848,2020-01-30 01:54:49+00:00,"@promo_BRI Min, kok Voucher nya belum diterima...",1,0,0,Twitter for Android,shandiroid,False,2010-02-06T04:42:16+00:00,200,337,2063,None,0,promo_bri,BRI,"[min, voucher, nya, terima, ya]"
4,1222660135664766976,2020-01-29 23:17:46+00:00,"Lakukan Inovasi dan Digitalisasi Layanan, BRI ...",0,0,0,Twitter Web App,KoranBUMN,False,2016-12-22T11:32:50+00:00,746,433,37476,"[kontakBRI, BANKBRI_ID, ojkindonesia, KemenBUMN]",4,promo_bri,BRI,"[laku, inovasi, digitalisasi, layan, bri, doro..."


In [11]:
def pandasToList(df, tidy_column):
    processedTweets = []
    for i in range(df.shape[0]):
        processedTweets.append((df.loc[i, tidy_column], None))
    return processedTweets
    
def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentimen) in preprocessedTrainingData:
        all_words.extend(words)
    
    wordlist = nltk.FreqDist(all_words)
    wordlist_filter = lambda word_freq: dict( (word,word_freq[word]) for word in word_freq if word_freq[word]>1 )

    return list(wordlist_filter(wordlist).keys())[10:]

def extract_features(tweet):
    tweet_words = set(tweet)
    features={}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

def preprocessTwits(df, column):
    df['temp_'] = df[column].apply(cleansingTwit)
    return pandasToList(df, 'temp_')

In [12]:
preprocessedTwit = pandasToList(df, 'tidy_twits')

In [18]:
import pickle
filename = 'nuSVC.sav'
NuSVC = pickle.load(open(filename, 'rb'))

In [20]:
filename = 'word_features.sav'
word_features = pickle.load(open(filename, 'rb'))

In [ ]:
df['prediction'] = pd.Series([NuSVC.classify(extract_features(tweet[0])) for tweet in preprocessedTwit])
df.head()

In [ ]:
df.to_pickle("sentimented.pkl")